# Week 3: Noise, Bias and Augmentation 

This week is all about looking at the practicalities of collecting data from the 'real world'
- thinking about 'sanity-checking' what we are  given
- thinking about how there might be biases in what we are given
- thinking about how there might be 'gaps' in what we are given

and how to deal with these.

## Example: revisiting the Palmer Penguins Dataset

This data has 344 records and seven features:
- Species (use this as the label to be predicted)
- Island
- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g
- sex
and this is what they look like

We can remind ourselves what the data set is like by looking at the [kaggle page](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data?resource=download&select=penguins_size.csv)

And this description is taken from  the original paper:

Each season, study nests, where pairs of adults were present, were individually marked and chosen before the onset of egg laying, and consistently monitored. When study nests were found at the one-egg stage, **both adults were captured** to obtain blood samples used for molecular sexing and SI analyses, and measurements of structural size and body mass. **At the time of capture, each adult penguin was quickly blood sampled** (∼1 ml) from the brachial vein using a sterile 3 ml syringe and heparinized infusion needle. Collected blood was stored in 1.5 ml micro-centrifuge tubes that were kept cool. In the field, a small amount of whole blood was smeared on clean filter paper stored in a 1.5 ml micro-centrifuge tube for molecular sexing. **Measurements of culmen length and depth (using dial calipers ±0.1 mm), right flipper (using a ruler ±1 mm), and body mass (using 5 kg±25 g or 10 kg±50 g Pesola spring scales and a weigh bag) were obtained to quantify body size variation**. After handling, individuals at study nests were further monitored to ensure the pair reached clutch completion, i.e., two eggs.






Gorman KB, Williams TD, Fraser WR (2014). Ecological sexual dimorphism and environmental variability within a community of Antarctic penguins (genus Pygoscelis). PLoS ONE 9(3):e90081. https://doi.org/10.1371/journal.pone.0090081

<div class="alert alert-block alert-warning" style="width:400px, text-color:black">     
    <h3>How accurately do you think you could measure a struggling penguin?</h3>
    </div>

### Another example: A medical dataset
Anonymised patient records and how long they survived


_citation_:
Raffa, J. (2016). Clinical data from the MIMIC-II database for a case study on indwelling arterial catheters (version 1.0). PhysioNet. https://doi.org/10.13026/C2NC7F.

Raffa J.D., Ghassemi M., Naumann T., Feng M., Hsu D. (2016) Data Analysis. In: Secondary Analysis of Electronic Health Records. Springer, Cham

In [2]:
import socket
import pandas as pd

thehost=socket.gethostname()
print(f'socket hostname is {thehost}')
if (socket.gethostname()=='csctcloud'): #on csctcloud
    datapath="/home/common/datasets"
elif (thehost[0:7]=='jupyter'): #on csctcloud
    datapath="~/shared/datasets"
else: #machine specific- this is for jim's development
    datapath = "../datasets"
    

mimic= pd.read_csv(datapath +"/mimic/mimic_iaccd_full_cohort_data_head.csv")


socket hostname is S-FET-QW50CYWDHW


In [3]:
mimic.head()

full_cohort_data
aline_flg icu_los_day hospital_los_day age      gender_num weight_first bmi         sapsi_first sofa_first service_unit service_num day_icu_intime day_icu_intime_num hour_icu_intime hosp_exp_flg icu_exp_flg day_28_flg mort_day_censored censor_flg sepsis_flg chf_flg afib_flg renal_flg liver_flg copd_flg cad_flg stroke_flg mal_flg resp_flg map_1st     hr_1st temp_1st    spo2_1st abg_count wbc_first hgb_first platelet_first sodium_first potassium_first tco2_first chloride_first bun_first creatinine_first po2_first pco2_first         iv_day_1
1         7.63        13               72.36841 1          75           29.91279129 15          9          SICU         1           Friday         6                  6               1            0           1          11.92             0          0          0       0        0         0         0        0       0          1       0        92          86     95.90000153 100      22        8.1       14.1      354            138          4.6             15         109            41        1.6              196       39                 2230.875
0         1.14        1                64.92076 0          55           20.12131169 NaN         5          MICU         0           Saturday       7                  17              0            0           0          731               1          0          0       0        0         0         0        0       0          0       0        86.66670227 85     97.59999847 100      1         NaN       NaN       NaN            NaN          NaN             NaN        NaN            NaN       NaN              NaN       NaN                     600
          2.86        5                36.5     0          70           27.11827188 16          5          MICU         0           Friday         6                  3               0            0           0          731               1          0          0       0        0         0         0        0       0          0       0        69.66670227 135    96.30000305 99       3         27        13.1      295            144          3.9             17         101            16        0.8              298       30              2086.800293
1         0.58        3                44.49191 0          NaN          NaN         21          7          SICU         1           Saturday       7                  4               1            1           1          0                 0          0          0       0        0         0         0        0       0          1       0        101         125    100.0999985 100      4         7.1       12.6      262            139          4.2             31         100            16        0.5              146       23                      NaN

From [this NHS website](https://www.nhs.uk/common-health-questions/lifestyle/what-is-the-body-mass-index-bmi/)

The body mass index (BMI) is a measure that uses your height and weight to work out if your weight is healthy.

<div class="alert alert-block alert-warning" style="width:400px, text-color:black">The BMI calculation divides an adult's weight in kilograms by their height in metres squared. 
</div>    For example, A BMI of 25 means 25kg/m2.
     
### How reasonable are the BMI values above?
- how much do you think they might vary during the day?

## The first take-home message: you can't always trust what you are told! 
You need to ask yourself:
- what determines the right precision for numerical values? the device or how it is used?
- rule of thumb for calculations: report to precision of least accurate part of calculation 

## Second Take-Home Message: 
## Taking a measurement/image/... repeatedly may give different results
- time of day for weight
- slight changes in lighting, camera angle/position, dirty lens
- varying background noise in sound recordings
- people get pay rises ...

But can we exploit this?

# Sources of Noise/ variation/ uncertainty in datasets

### Source 1: How we sample items from the underlying distribution of things (the 'world') to measure
 - Does distribution of items we select match the underlying distribution  
   e.g. are we sampling from hospital patients, who may be more likely to have a disease  
 - Do we need to take country/geography /spatial factors into account       
 - Is  the distribution of types of things in the world is changing over time  
   e.g. proportion of people testing positive for a disease 
 - Are the measurements for different subsets of data are changing over time.   
   e.g. different variants of a disease with slightly different symptoms  
    cars getting heavier (electric batteries)
    different groups of people living longer

   
 **These are potential sources of unethical/ unwanted bias in our samples**  
   We may be able to deal with these using things like Generative Adversarial Networks
   - there's **lots** of work on using GANs to generate data for training autonmous vehicles
   - although [a recent paper](https://www.sciencedirect.com/science/article/pii/S0004370221002034#se0060) suggest this may be problematic when it comes to addressing bias around human records and images

**Law and Ethics** 
The Information Commissioner's Office [AI and Data Protection Risk Toolkit](https://ico.org.uk/for-organisations/uk-gdpr-guidance-and-resources/artificial-intelligence/guidance-on-ai-and-data-protection/ai-and-data-protection-risk-toolkit/) 
- applies to anyone working in the UK and training a model on people's data 
- lays clear obligations to
  - assess risk that AI disadvantages people according to protected characteristics
  - measure risk of 'privacy leakage' (more on this another week)
   

### Source 2: How we measure/capture/represent the samples we take to create a dataset
 - If can identify valid transformations we may be able use these to generalise  
   **e.g. Data Augmentation = today's practical** 
   
   Question: Is this a link between 'computational AI' and semantics?
   

### Source 3: How we divide our dataset into training, validation and test sets
 - This is why we should use techniques like N-fold cross validation if we have limited data
 - There is ongoing debate in the ML community about whether it is better to do, for example,  
   one set of 10-fold cross validation (i.e. split data into ten parts randomly once),  
   or 5 X 2-fold cross-validation ( 5 repeats of randomised equal-sized train-test splits)
  - A single train/test split is not adequate unless you have **lots** of **representative** data 

## and then on top of that ...
### Source 4: the variation that comes with some ML algorithms
- random weight initialisation
- stochastic routines in (for example) evolving trees (Genetic Programming), or Random Forests

# So what is data augmentation all about?

1. Come up with some valid transformations for your data
  - adding noise to measurements
  - accounting for different camera positions / lighting
  - using synonyms in text
  - changing background noise in images or sound
  

2. Apply the transformations to copies of your existing labelled data to make more examples   
  (keeping the labels the same)  
   Either use them to generate:
     - a fixed extra set of more training examples 
     - new items for each epoch of training (obviously  Neural Nets only)
  

3. train with the extra data (or data generators) in the hope of reducing over-fitting   
  by taking account of the 'measurement' effect from above

## Possible problems?

- can replicate existing bias in the way we sample things to measure/represent
- what if your samples were outliers?  
  adding perturbations might take you even further from 'normal' 
  
-   What do we mean by 'valid'? who gets to decide?
  
Good discussion of pros and cons for different types of data in [this blog](https://www.datacamp.com/tutorial/complete-guide-data-augmentation)